In [1]:
!pip install torch torchvision
!pip install scikit-learn
!pip install pillow

In [15]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

In [3]:
fruit_dir = '/kaggle/input/dataset/Fruits_Vegetables_Dataset(12000)/Fruits'
veg_dir = '/kaggle/input/dataset/Fruits_Vegetables_Dataset(12000)/Vegetables'

In [4]:
fruit_classes = ['FreshApple', 'RottenApple', 'FreshBanana', 'RottenBanana','FreshMango','RottenMango','FreshOrange','RottenOrange','FreshStrawberry','RottenStrawberry']  # List fruit classes
veg_classes = ['FreshCarrot', 'RottenCarrot', 'FreshTomato', 'RottenTomato', 'FreshCucumber','RottenCucumber','FreshPotato','RottenPotato','FreshBellpepper','RottenBellpepper' ]  # List vegetable classes

In [5]:
def load_data(data_dir, classes):
    images = []
    labels = []

    for class_name in classes:
        class_path = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = Image.open(img_path)
            img = img.resize((224, 224))  # Resize images to match pre-trained model input size
            img = transforms.ToTensor()(img)  # Convert to tensor

            # Check if image has correct number of channels (RGB)
            if img.size(0) == 3:
                images.append(img)
                labels.append(classes.index(class_name))

    images = torch.stack(images)
    labels = np.array(labels)

    return images, labels

In [6]:
veg_images, veg_labels = load_data(veg_dir, veg_classes)

In [7]:
fruit_images, fruit_labels = load_data(fruit_dir, fruit_classes)

In [8]:
images = torch.cat((fruit_images, veg_images), dim=0)
labels = np.concatenate((fruit_labels, veg_labels))


In [9]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

Using Different models to train and test.

In [13]:

googlenet = models.googlenet()
googlenet_features = []
print("Done downloading...")
with torch.no_grad():
    for img in X_train:
        features = googlenet(img.unsqueeze(0))
        googlenet_features.append(features.squeeze().numpy())
googlenet_features = np.array(googlenet_features)
print("Done Extracting...")



Done downloading...


In [15]:
googlenet.eval()

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [14]:
densenet = models.densenet201()
densenet_features = []
print("Done Downloading....")
with torch.no_grad():
    for img in X_train:
        features = densenet(img.unsqueeze(0))
        densenet_features.append(features.squeeze().numpy())
densenet_features = np.array(densenet_features)
print("Done Extracting....")

Done Downloading....
Done Extracting....


In [16]:
densenet.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [21]:
resnext = models.resnext101_32x8d()
resnext_features = []
print("Done Downloading....")
with torch.no_grad():
    for img in X_train:
        features = resnext(img.unsqueeze(0))
        resnext_features.append(features.squeeze().numpy())
resnext_features = np.array(resnext_features)
print("Done Extracting....")

Done Downloading....
Done Extracting....


In [22]:
resnext.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [23]:
fused_features = np.concatenate((googlenet_features, densenet_features, resnext_features), axis=1)
pca = PCA(n_components=50)
reduced_features = pca.fit_transform(fused_features)

In [26]:
googlenet_test_features = []
densenet_test_features = []
resnext_test_features = []
print("Feature Setting....")
with torch.no_grad():
    for img in X_test:
        googlenet_test_features.append(googlenet(img.unsqueeze(0)).squeeze().numpy())
        densenet_test_features.append(densenet(img.unsqueeze(0)).squeeze().numpy())
        resnext_test_features.append(resnext(img.unsqueeze(0)).squeeze().numpy())
    print("Done")



Feature Setting....
Done


In [27]:
googlenet_test_features = np.array(googlenet_test_features)
densenet_test_features = np.array(densenet_test_features)
resnext_test_features = np.array(resnext_test_features)

fused_test_features = np.concatenate((googlenet_test_features, densenet_test_features, resnext_test_features), axis=1)
reduced_test_features = pca.transform(fused_test_features)



In [28]:
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(reduced_features, y_train)
svm_clf = SVC(kernel='poly', degree=3)
svm_clf.fit(reduced_features, y_train)
bagging_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100)
bagging_clf.fit(reduced_features, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100)

In [29]:
lda_preds = lda_clf.predict(reduced_test_features)
svm_preds = svm_clf.predict(reduced_test_features)
bagging_preds = bagging_clf.predict(reduced_test_features)

In [30]:
print("LDA Performance:")
print(f"Accuracy: {accuracy_score(y_test, lda_preds)}")
print(f"Precision: {precision_score(y_test, lda_preds, average='macro')}")
print(f"Recall: {recall_score(y_test, lda_preds, average='macro')}")
print(f"F1-score: {f1_score(y_test, lda_preds, average='macro')}")
print(f"MCC: {matthews_corrcoef(y_test, lda_preds)}")

print("\nSVM Performance:")
print(f"Accuracy: {accuracy_score(y_test, svm_preds)}")
print(f"Precision: {precision_score(y_test, svm_preds, average='macro')}")
print(f"Recall: {recall_score(y_test, svm_preds, average='macro')}")
print(f"F1-score: {f1_score(y_test, svm_preds, average='macro')}")
print(f"MCC: {matthews_corrcoef(y_test, svm_preds)}")

print("\nBagging Performance:")
print(f"Accuracy: {accuracy_score(y_test, bagging_preds)}")
print(f"Precision: {precision_score(y_test, bagging_preds, average='macro')}")
print(f"Recall: {recall_score(y_test, bagging_preds, average='macro')}")
print(f"F1-score: {f1_score(y_test, bagging_preds, average='macro')}")
print(f"MCC: {matthews_corrcoef(y_test, bagging_preds)}")

LDA Performance:
Accuracy: 0.1195421788893599
Precision: 0.2530981577906026
Recall: 0.11208033982927787
F1-score: 0.05973454279616328
MCC: 0.017119633644311544

SVM Performance:
Accuracy: 0.16066129715981348
Precision: 0.22882490884171253
Recall: 0.1620445844144006
F1-score: 0.11763178157000535
MCC: 0.0760302642109134

Bagging Performance:
Accuracy: 0.16362865621025857
Precision: 0.1532030315421902
Recall: 0.1687619269557984
F1-score: 0.13991551550331746
MCC: 0.07484763660404382


In [1]:
import os
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [2]:

class CustomDataset(Dataset):
    def __init__(self, data_dir, classes, transform=None):
        self.data_dir = data_dir
        self.classes = classes
        self.transform = transform
        self.images, self.labels = self.load_data()

    def load_data(self):
        images = []
        labels = []

        for class_name in self.classes:
            class_path = os.path.join(self.data_dir, class_name)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = Image.open(img_path)
                img = img.resize((224, 224))  # Resize images to match pre-trained model input size

                # Check if image has correct number of channels (RGB)
                if img.mode == 'RGB':
                    images.append(img)
                    labels.append(self.classes.index(class_name))

        return images, labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [3]:

transform = transforms.Compose([
    transforms.ToTensor(),
])



In [4]:

fruit_dir = '/kaggle/input/dataset/Fruits_Vegetables_Dataset(12000)/Fruits'
veg_dir = '/kaggle/input/dataset/Fruits_Vegetables_Dataset(12000)/Vegetables'
fruit_classes = ['FreshApple', 'RottenApple', 'FreshBanana', 'RottenBanana','FreshMango','RottenMango','FreshOrange','RottenOrange','FreshStrawberry','RottenStrawberry']
veg_classes = ['FreshCarrot', 'RottenCarrot', 'FreshTomato', 'RottenTomato', 'FreshCucumber','RottenCucumber','FreshPotato','RottenPotato','FreshBellpepper','RottenBellpepper' ]



In [5]:

fruit_dataset = CustomDataset(fruit_dir, fruit_classes, transform=transform)
veg_dataset = CustomDataset(veg_dir, veg_classes, transform=transform)


In [6]:

fruit_loader = DataLoader(fruit_dataset, batch_size=len(fruit_dataset))
veg_loader = DataLoader(veg_dataset, batch_size=len(veg_dataset))


In [7]:

fruit_images, fruit_labels = next(iter(fruit_loader))
veg_images, veg_labels = next(iter(veg_loader))


In [ ]:

images = torch.cat((fruit_images, veg_images), dim=0)
labels = np.concatenate((fruit_labels, veg_labels))


In [9]:

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [12]:

def extract_features(model, image):
    img_tensor = transforms.ToTensor()(image)  # Convert image to tensor
    with torch.no_grad():
        features = model(img_tensor.unsqueeze(0)).squeeze().numpy()  # Extract features
    return features

In [18]:
googlenet = models.googlenet()
densenet = models.densenet201()
resnext = models.resnext101_32x8d()

In [ ]:

train_features = np.concatenate((googlenet_features_train, densenet_features_train, resnext_features_train), axis=1)


In [ ]:

googlenet_features_train = np.array([extract_features(googlenet, img) for img in X_train])
densenet_features_train = np.array([extract_features(densenet, img) for img in X_train])
resnext_features_train = np.array([extract_features(resnext, img) for img in X_train])


In [ ]:

svm_clf = SVC(kernel='poly', degree=3)
svm_clf.fit(reduced_train_features, y_train)

In [ ]:


def detect_fruits_and_vegetables(model, image):
    # Extract features from the image
    features = extract_features(model, image)
    
    # Reduce features using PCA
    reduced_features = pca.transform(features.reshape(1, -1))

    # Make prediction using SVM classifier
    prediction = svm_clf.predict(reduced_features)

    # Return the predicted class
    return prediction[0]

# Release memory
del fruit_images, fruit_labels, veg_images, veg_labels, images, labels, X_train, X_test, y_train, y_test
torch.cuda.empty_cache()


In [ ]:
# Assuming your test images are located in a directory named "test_images"
test_image_dir = "/kaggle/input/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset(12000)/Fruits/FreshBanana/"

# Perform real-time detection on test images
for i, filename in enumerate(os.listdir(test_image_dir)):
    image_path = os.path.join(test_image_dir, filename)
    test_image = Image.open(image_path)

    predicted_class = detect_fruits_and_vegetables(googlenet, test_image)
    print(f"Test Image {i+1}: Predicted Class - {predicted_class}")
